In [1]:
import json
import boto3

In [4]:
# Let's take a look under the hood at what's happening with our logs. 
# First we'll take a look at what queues we have available 

sqs = boto3.client('sqs')
queues = sqs.list_queues()
display(queues)

{'QueueUrls': ['https://us-west-2.queue.amazonaws.com/583449068983/aws_splunk_main_queue',
  'https://us-west-2.queue.amazonaws.com/583449068983/aws_splunk_queue_deadletter'],
 'ResponseMetadata': {'RequestId': 'd477649d-30b4-5521-86f3-32344a7d98c3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd477649d-30b4-5521-86f3-32344a7d98c3',
   'date': 'Mon, 27 Jan 2020 23:55:53 GMT',
   'content-type': 'text/xml',
   'content-length': '438'},
  'RetryAttempts': 0}}

In [10]:
# Now let's iterate through those queues and print the URL of the 
# "aws_splunk_main_queue" queue.
queue_url = [q for q in queues['QueueUrls'] if q.endswith('aws_splunk_main_queue')][0]
display(queue_url)

'https://us-west-2.queue.amazonaws.com/583449068983/aws_splunk_main_queue'

In [26]:
# Pull a message from the queue and see what it looks like.
# You might have to try a few times to get something similar to this one
sqs.list_queues()
message = sqs.receive_message(QueueUrl=queue_url, AttributeNames=["All"])
display(message)

{'ResponseMetadata': {'RequestId': 'f9e92e41-a1a5-58a9-a63a-78e820bbdb48',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f9e92e41-a1a5-58a9-a63a-78e820bbdb48',
   'date': 'Tue, 28 Jan 2020 00:23:04 GMT',
   'content-type': 'text/xml',
   'content-length': '240'},
  'RetryAttempts': 0}}

In [ ]:
"""FIXME: Not getting any messages with "Messages" like I did before. 
Not sure why - permissions? No messages to grab?

```python
message = sqs.receive_message(QueueUrl=queue_url)
# Output will be something like:
# {'Messages': [{'MessageId': '7de4a3d9-2868-4ece-aeb3-a1460eefb6b3',
#    'ReceiptHandle': 'AQEBCfZqk25k+7eRX8VM3yRQyEcu0s5zSv2VcI853B6uYJebyg+K8m7HqrhnRZD6syQlYFc7z7cDAh6f6sJ3u+lcTSK3lLdkzaLeUDwdQHgItr0KPy1dNQGkH/WK/Dt/0BANXCvFiyzvOMvTkl9JiYhEzYH9FRbGwW63xUiEsNxYKIr1h73iZ8KCdWpPElGWrY7/l27KOrj4nM0eWWLCEEIInGoG3JEX89X0QCI5AA4YCx/c1yWDx/a/qLYge5DFmTe2WEEDGj3j91x/aQATRC7cunaKKmUp1lviuq0HBexQFvneH36CHq+7a6WSoreHb8CGDpvD3sa8+bLn3Ng6EwORL+ZMh1lQ7aF3YzHUcC8kkm3QHjpv92INlw20ZsrUCMeIrD/wJibluv30Ptc5VUtUlw==',
#    'MD5OfBody': 'c5b1d12d4bf4913856579c8c29a41df2',
#    'Body': '{\n  "Type" : "Notification",\n  "MessageId" : "d4ef62a2-b761-565f-ac03-34365d35d874",\n  "TopicArn" : "arn:aws:sns:us-west-2:ACCOUNT_ID:bsides-topic",\n  "Message" : "{\\"s3Bucket\\":\\"bsides-trail-20200112083857693900000001\\",\\"s3ObjectKey\\":[\\"prefix/AWSLogs/ACCOUNT_ID/CloudTrail/us-west-2/2020/01/12/ACCOUNT_ID_CloudTrail_us-west-2_20200112T0930Z_XNcKI5JFGkzUeu0m.json.gz\\"]}",\n  "Timestamp" : "2020-01-12T09:34:12.116Z",\n  "SignatureVersion" : "1",\n  "Signature" : "Smtdm5Cai/ortbXR7wk1Vv4kwy3RlTelAH2UO4cLx1/tKukQEA/fqyLASZ7A3VkZlJVbSyDJ5n93hBHvY8+mXt5ip39r6O14D3iz7zh5SFUHXJgX8Csr1W9azJTfoiefbquH3bhyceLwQxrnw/FBi1LXkFgDNDsZj753QM0M192TQ1JTxNu2hoph5G++pIRXVVrg7pTYV8xxzJV4C/NvCK0vUnq3BnAnbSHVgWx7IIVlY3i9vmzI7wd7mpc2rDNiJPNHMdaUczCZU31u8WyGasA6EicyhvizUz6AxHfykNcNRPnz/QnBCk+8tnltAf7uF1LlI1ICrwIzbRkQ0gRHcg==",\n  "SigningCertURL" : "https://sns.us-west-2.amazonaws.com/SimpleNotificationService-a86cb10b4e1f29c941702d737128f7b6.pem",\n  "UnsubscribeURL" : "https://sns.us-west-2.amazonaws.com/?Action=Unsubscribe&SubscriptionArn=arn:aws:sns:us-west-2:ACCOUNT_ID:bsides-topic:c7d39b72-b232-4e5b-8bde-a8a0469c0413"\n}'}],
#  'ResponseMetadata': {'RequestId': '7daac050-2003-543d-b2ea-96c4d2101de6',
#   'HTTPStatusCode': 200,
#   'HTTPHeaders': {'x-amzn-requestid': '7daac050-2003-543d-b2ea-96c4d2101de6',
#    'date': 'Sun, 12 Jan 2020 10:07:15 GMT',
#    'content-type': 'text/xml',
#    'content-length': '2198'},
#   'RetryAttempts': 0}}
```

Wow, that is pretty terrible. If we dig in a little bit, it looks like the message body is still raw JSON. Let's pull it out:

```python
message_body = json.loads(message['Messages'][0]['Body'])

message_body['Message']
print(message_body)


# {'Type': 'Notification',
#  'MessageId': 'd4ef62a2-b761-565f-ac03-34365d35d874',
#  'TopicArn': 'arn:aws:sns:us-west-2:ACCOUNT_ID:bsides-topic',
#  'Message': '{"s3Bucket":"bsides-trail-20200112083857693900000001","s3ObjectKey":["prefix/AWSLogs/ACCOUNT_ID/CloudTrail/us-west-2/2020/01/12/ACCOUNT_ID_CloudTrail_us-west-2_20200112T0930Z_XNcKI5JFGkzUeu0m.json.gz"]}',
#  'Timestamp': '2020-01-12T09:34:12.116Z',
#  'SignatureVersion': '1',
#  'Signature': 'Smtdm5Cai/ortbXR7wk1Vv4kwy3RlTelAH2UO4cLx1/tKukQEA/fqyLASZ7A3VkZlJVbSyDJ5n93hBHvY8+mXt5ip39r6O14D3iz7zh5SFUHXJgX8Csr1W9azJTfoiefbquH3bhyceLwQxrnw/FBi1LXkFgDNDsZj753QM0M192TQ1JTxNu2hoph5G++pIRXVVrg7pTYV8xxzJV4C/NvCK0vUnq3BnAnbSHVgWx7IIVlY3i9vmzI7wd7mpc2rDNiJPNHMdaUczCZU31u8WyGasA6EicyhvizUz6AxHfykNcNRPnz/QnBCk+8tnltAf7uF1LlI1ICrwIzbRkQ0gRHcg==',
#  'SigningCertURL': 'https://sns.us-west-2.amazonaws.com/SimpleNotificationService-a86cb10b4e1f29c941702d737128f7b6.pem',
#  'UnsubscribeURL': 'https://sns.us-west-2.amazonaws.com/?Action=Unsubscribe&SubscriptionArn=arn:aws:sns:us-west-2:ACCOUNT_ID:bsides-topic:c7d39b72-b232-4e5b-8bde-a8a0469c0413'}
```

That's looking better, but the `Message` field seems to be more JSON buried inside that JSON! Let's parse it out:
```python
message_body_message = json.loads(message_body['Message'])

# {'s3Bucket': 'bsides-trail-20200112083857693900000001',
#  's3ObjectKey': ['prefix/AWSLogs/ACCOUNT_ID/CloudTrail/us-west-2/2020/01/12/ACCOUNT_ID_CloudTrail_us-west-2_20200112T0930Z_XNcKI5JFGkzUeu0m.json.gz']}
```

Now we've got something useful. SNS is essentially providing information about where files are being written to in our S3 bucket. Let's grab that file and see what's in it:

```python
s3 = boto3.client("s3")
s3_bucket = message_body_message['s3Bucket']
s3_object_key = message_body_message['s3ObjectKey'][0]
s3.download_file(s3_bucket, s3_object_key, 'logs.json.gz')
```

This will download a file named `logs.json.gz` to our local directory. Unzip it with `gunzip logs.json.gz` and you can see the raw log events. These are the same as what will be ingested in Splunk.
"""